In [1]:
import pandas as pd
from urllib.parse import quote
from google.colab import files
import numpy as np
import datetime
import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time

In [2]:
file_path = "/content/1923_dramalist.csv"
drama_list = pd.read_csv(file_path, index_col = 0)
drama_list

,작품명,극본,연출,방송 기간,방송,방송사,유형,UHD,방송.1,시청,비고
0,"신비아파트 외전: 기억, 하리 2",이인혜,박용진,"2019.02.15, 2019.04.05",8,투니버스,금,NaN,NaN,(),1회 연장
1,너를 싫어하는 방법,"유소원, 정수현",오관진,"2019.04.1, 2019.04.2",2,"JTBC2,4","(월화), 단막",NaN,NaN,(),8회 연장
2,아름다운 세상,김지우,박찬홍,"2019.05.25, 2019.04.05",16,JTBC,금토,NaN,NaN,(),8회 연장
3,국민 여러분!,한정훈,김정현,"2019.05.28, 2019.04.01",36 (18),KBS2,월화,NaN,NaN,(),8회 연장
4,날 녹여주오,백미경,신우철,"2019.09.28, 2019.11.17",16,tvN,토일,NaN,NaN,(),사전제작
...,...,...,...,...,...,...,...,...,...,...,...
415,가슴이 뛴다,"정승주, 김하나","이민수, 이현석",2023.6.26,16,KBS2,월화,NaN,NaN,(),사전제작
416,기적의 형제,김지우,박찬홍,2023.6.28,16,JTBC,수목,NaN,NaN,(),사전제작
417,닥터 차정숙,정여랑,김대진,"2023.6.4, 2023.4.15",16,JTBC,토일,NaN,NaN,(),사전 제작
418,이로운 사기,한우주,이수현,"2023.7.18, 2023.5.29",16,tvN,월화,NaN,NaN,(),사전제작


# 크롤링

## 함수

In [3]:
# 드라마 제목을 위키피디아에 검색해 html데이터를 읽어옴.
def read_rating(title):
  rate = pd.DataFrame()
  front_url = "https://ko.wikipedia.org/wiki/"
  error= None

  try:
    encoded_url = quote(front_url + title + "_(대한민국의 드라마)", safe=':/_')
    rate = func1(encoded_url)
    rate['작품명'] = title
    print('1번째')

  except:
    try:
      encoded_url = quote(front_url + title + "_(드라마)", safe = ':_/')
      rate = func1(encoded_url)
      rate['작품명'] = title
      print('2번째')

    except:
      try:
        encoded_url = quote(front_url + title, safe = ':/')
        rate = func1(encoded_url)
        rate['작품명'] = title
        print('3번째')
      except Exception as e:
        error = title
        print(e)
        pass

  return rate, error


# 두 열의 이름이 같으면 열을 하나로 합치되 값은 뒤의 열 사용
def merge_duplicate_columns(df):
    duplicated_cols = df.columns[df.columns.duplicated(keep=False)]
    for col in duplicated_cols:
        df[col] = df.loc[:, col].iloc[:, -1]
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    return df



def func1(encoded_url):
  rate = pd.DataFrame()
  table_matching_list = ['제1회', '제1화', '1화', '1회']
  tables = pd.read_html(encoded_url)

  for table in tables:
    if table.iloc[0,0] in table_matching_list:
      rate = table


  if rate.columns.nlevels == 3:
    year = rate.columns[0][0]
    rate.columns = rate.columns.map(lambda x: f"{x[1]}, {x[2]}" if len(x) == 3 else x)
    rate['방영연도'] = year
  elif rate.columns.nlevels == 2:
    year = rate.columns[0][0]
    rate.columns = rate.columns.map(lambda x: f"{x[1]}" if len(x) == 2 else x)
    rate['방영연도'] = year

  # [숫자] 형식의 글자를 전부 제거
  # pattern = r'\[\d+\]'
  # rate.columns =  [re.sub(pattern, '', str(col)) for col in rate.columns]


  # Or(|) 로 묶은 단어들이 있는 열은 삭제
  rate = rate.loc[:, ~rate.columns.str.contains('TNms', case = False)]
  rate = rate.loc[:, ~rate.columns.str.contains('수도권')]
  rate = rate.loc[:, ~rate.columns.str.contains('부 제|부제')]
  rate = rate.loc[:, ~rate.columns.str.contains('제목')]
  rate = rate.loc[:, ~rate.columns.str.contains('재방송')]

  # '전국'이라는 단어가 포함된 열의 이름을 '시청률'으로 변경
  for col in rate.columns:
    if '전국' in col:
      rate.rename(columns={col: '시청률'}, inplace=True)


  rate = rate.loc[:, ~rate.columns.str.contains('Unnamed', case = False)]

  # 2. 쉼표를 기준으로 두 단어가 같은 경우 한 단어로.
  for col in rate.columns:
    words = col.split(',')
    if len(words) > 1 and words[0].strip() == words[1].strip():
      rate.rename(columns={col: words[0]}, inplace=True)
  rate = merge_duplicate_columns(rate) # 두 열의 이름이 같으면 하나의 열로

  # 3. 시청률 이란 단어가 포함된 열의 이름을 시청률로 변경
  for col in rate.columns:
    if '시청률' in col:
      rate.rename(columns={col: '시청률'}, inplace=True)

  # 4. 회차 가 들어있으면 회차로 변경
  for col in rate.columns:
    if '회차' in col:
      rate.rename(columns={col: '회차'}, inplace=True)

  # 5. 방송일이 들어있으면 방송일자로 변경
  for col in rate.columns.str.replace(' ', '').tolist():
    if '방송일' in col:
      rate.rename(columns={col: '방송일자'}, inplace=True)

  return rate



## 크롤링 실행 / 열정리

In [4]:
drama_list = drama_list['작품명'].tolist()
drama_list.extend(['에이틴', '에이틴 2', '멀리서 보면 푸른 봄'])

In [5]:
# 크롤링 코드 (8분 정도 소요)
error_list = []
rating_df = pd.DataFrame()

for title in tqdm(drama_list):
  rate, error = read_rating(title)
  print(title)
  error_list.append(error)
  rating_df = pd.concat([rating_df, rate], ignore_index = True)
  time.sleep(1) # http 429 error: 서버에 너무 많은 요청을 빠르게 하면 생기는 에러를 방지하기 위한 코드

error_list = [x for x in error_list if x is not None]


  0%|          | 0/423 [00:00<?, ?it/s]

3번째
신비아파트 외전: 기억, 하리 2


  0%|          | 1/423 [00:02<15:49,  2.25s/it]

HTTP Error 404: Not Found
너를 싫어하는 방법


  0%|          | 2/423 [00:04<14:25,  2.05s/it]

...

100%|█████████▉| 421/423 [12:42<00:03,  1.64s/it]

3번째
에이틴 2


100%|█████████▉| 422/423 [12:44<00:01,  1.69s/it]

2번째
멀리서 보면 푸른 봄


100%|██████████| 423/423 [12:46<00:00,  1.81s/it]


In [6]:
print(rating_df.columns.tolist())
print(error_list)


['회차', '방송 날짜 (투니버스)', '작품명', '방송일자', '시청률', '방영연도', '방송 일자', 'AGB[2]', '방송일, 2021년']
['너를 싫어하는 방법', '각색은 이미 시작됐다', '개같다 거지같다 아름답다', '에이틴 / 에이틴 시즌 2', '아이를 찾습니다', '멀리서 보는 푸른 봄']


In [18]:
# 신비아파트 제거
rating_df = rating_df[rating_df['방송 날짜 (투니버스)'].isna()]

# 열 정리
rating_df['방송일자'] = rating_df['방송일자'].fillna(rating_df['방송 일자'])
rating_df['방송일자'] = rating_df['방송일자'].fillna(rating_df['방송일, 2021년'])
rating_df['시청률'] = rating_df['시청률'].fillna(rating_df['AGB[2]'])
rating_df = rating_df.drop(['방송 날짜 (투니버스)', '방송 일자', 'AGB[2]', '방송일, 2021년'], axis=1)


# 전처리

In [64]:
# rating_df 는 원본으로 냅두고 x로 조작 -> 다시 처음부터 크롤링해야하는 부담 경감
x = rating_df.copy()

In [65]:
# 회차 처리
x = x[~x['회차'].isnull()]
x = x[~x['회차'].str.contains('스페셜')]

x['회차'] = x['회차'].str.extract('(\d+)', expand=False) # 숫자 형식만 추출하고 숫자가 없으면 Nan 반환(expand)
x = x[~x['회차'].isnull()]  # 스페셜 1부/2부 이런 문제들을 제거하기 위해 na값 지우는 것을 이중으로 배치(na가 있으면 contain 작동 x)

In [66]:
# 방송일자
## 방송일자에서 월, 년, - 을 /로 바꿔주고, 공백, '일' 제거
x['방송일자'] = x['방송일자'].str.replace('월', '/').str.replace(' ', '').str.replace('일', '').str.replace('-', '/').str.replace('년', '/')
## /를 기준으로 방송일자를 잘라서, 자른 것의 길이가 3인 경우와 2인 경우 각각 다르게 월/일 데이터에 들어가도록 조정
x['월'] = x['방송일자'].apply(lambda date: date.split('/')[1] if pd.notna(date) and len(date.split('/')) == 3 else date.split('/')[0] if pd.notna(date) else np.nan)
x['일'] = x['방송일자'].apply(lambda date: date.split('/')[2] if pd.notna(date) and len(date.split('/')) == 3 else date.split('/')[1] if pd.notna(date) else np.nan)

## 나중에 숫자 조작 편하도록 숫자 형식으로 변환. 병합으로 인해 2000 이런 값도 있는데 이런 행들도 삭제
x['월'] = pd.to_numeric(x['월'])
x = x.drop(x[x['월'] > 12].index)

# 방영연도 처리
## 20xx년 처럼 되어있으므로 숫자 형식만 추출
x['방영연도'] = x['방영연도'].str.extract('(\d+)', expand=False)
## 아까 / 로 나눴을 때 길이가 3인 경우, 첫번째는 연도이므로 연 열에 삽입
x['연'] = x['방송일자'].apply(lambda date: date.split('/')[0] if pd.notna(date) and len(date.split('/')) == 3 else np.nan)

x['방영연도'] = x['방영연도'].fillna(x['연'])   # 연 열을 채웠으니 그것으로 방영연도의 공란을 채움
x.loc[x['방영연도'].isna(), '방영연도'] = 0     # 일단 방영연도의 빈 칸은 0으로 전환
x['방영연도'] = pd.to_numeric(x['방영연도'])    # 숫자형식 변환


# 해가 넘어가는 드라마 처리
list_12 = x[x['월'] == 12]['작품명'].unique().tolist()
list_1 = x[x['월'] == 1]['작품명'].unique().tolist()
year_list = list(set(list_12) & set(list_1)) ## 하나의 작품명에 1월과 12월이 동시에 있으면 해가 넘어갔다고 판단. 그런 드라마 리스트 추출


x.loc[x['작품명'] == '설강화', '방영연도'] = 2021


for title in year_list:
  k = x[x['작품명'] == title]                               # year_list에 있는 각 작품명별로 데이터를 호출하고,
  start = k['월'].tolist()[0]                               # 시작월을 변수로 지정.
  k.loc[k['월'] < start, '방영연도'] += 1                   # 시작월보다 월이 작다면 해가 넘어간 것이므로 방영연도에 +1
  x.loc[x['작품명'] == title, '방영연도'] = k['방영연도']   # 원래 데이터의 방영연도 열을 바꾸기


In [67]:
# '방송연도', '월', '일' 열을 문자열 형태로 변환
x['방영연도'] = x['방영연도'].astype(str)
x.loc[~x['월'].isna(), '월'] = x.loc[~x['월'].isna(), '월'].astype(int).astype(str) # 소숫점 형태로 되어있어서 int로 바꾸고 str로 바꿔야함
x.loc[~x['일'].isna(), '일'] = x.loc[~x['일'].isna(), '일'].astype(int).astype(str)

# 방송일자 생성
x['방송일자'] = x['방영연도'] + '-' + x['월'] + '-' + x['일']

# 방송일자를 dt 형식으로 변환
x['방송일자'] = pd.to_datetime(x['방송일자'], format='%Y-%m-%d', errors='coerce')
x = x.drop('연', axis = 1)

In [68]:
# 시청률
x['시청률'] = x['시청률'].str.replace(",", "").str.replace('%', '').str.replace('\[\d+\]', '', regex=True).str.replace('-', '')
x['시청률'] = pd.to_numeric(x['시청률'])
x.loc[x['시청률'] > 50, '시청률'] = [1.1167, 6.39] # 콤마가 이상하게 찍혀있는 데이터라 임의 수정

In [70]:
# 연속방송 반영 전의 데이터
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10489 entries, 8 to 10724
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회차      10489 non-null  object        
 1   작품명     10489 non-null  object        
 2   방송일자    10489 non-null  datetime64[ns]
 3   시청률     10240 non-null  float64       
 4   방영연도    10489 non-null  object        
 5   월       10489 non-null  object        
 6   일       10489 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 655.6+ KB


In [71]:
#연속방송 반영
## 연속방송회차 저장
grouped = x.copy().groupby(['작품명', '방송일자'])
mean_arg_count = grouped['회차'].aggregate(lambda y: len(y)).reset_index()

# grouping 시 숫자 데이터만 적용되므로 형식 수정
x['방영연도'] = x['방영연도'].astype(int)
x['월'] = x['월'].astype(int)
x['일'] = x['일'].astype(int)

# 작품명/방송일자로 grouping하고 cumcount로 회차를 1화부터 재배정
x = x.groupby(['작품명', '방송일자']).mean().reset_index() # 평균 과정에서 na는 무시
x['회차'] = x.groupby('작품명').cumcount() + 1
x['연속방송'] = mean_arg_count['회차']

x = x.reset_index(drop = True) # groupby 했던 것들 해제

x['방영연도'] = x['방영연도'].astype(int)
x['월'] = x['월'].astype(int)
x['일'] = x['일'].astype(int)

<ipython-input-71-75ba59447058>:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['작품명', '방송일자']).mean().reset_index() # 평균 과정에서 na는 무시


In [82]:
# 금년 7월 23일까지의 자료만 반영
x = x[x['방송일자'] <= '2023-07-23']

In [83]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8599 entries, 0 to 8739
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   작품명     8599 non-null   object        
 1   방송일자    8599 non-null   datetime64[ns]
 2   시청률     8550 non-null   float64       
 3   방영연도    8599 non-null   int64         
 4   월       8599 non-null   int64         
 5   일       8599 non-null   int64         
 6   회차      8599 non-null   int64         
 7   연속방송    8599 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 604.6+ KB


In [85]:
# 2023년 7월 1일 이전 시청률 누락 작품 목록. 2023년 7월 19일 이후는 미래라서 아직 없음.
# '사생활', '엑스가리온', '영혼수선공', '인서울 - 내가 독립하는 유일한 방법', 투니버스의 신비아파트 시리즈
#  엑스가리온/사생활/신비아파트는 전부, 나머지는 일부 시청률 누락
x.loc[(x['시청률'].isna()) & (x['방송일자'] < '2023-07-01'), '작품명'].unique()

array(['사생활', '엑스가리온', '영혼수선공', '인서울 - 내가 독립하는 유일한 방법'], dtype=object)

In [86]:
x.to_csv('/content/전체시청률.csv', index = False, encoding = 'utf-8-sig')